In [14]:
import sys
import spotipy
import pprint
import yaml
import spotipy.util as util
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

## Spotipy Credentials and API Calls

In [6]:
# Get Spotipy credentials from config
def load_config():
    stream = open('config.yaml')
    user_config = yaml.load(stream, Loader=yaml.FullLoader)
    return user_config

user_config = load_config()

In [16]:
# Initialize Spotify API token
token = util.prompt_for_user_token(user_config['username'], scope='user-read-recently-played', client_id=user_config['client_id'], client_secret=user_config['client_secret'], redirect_uri=user_config['redirect_uri'])
sp = spotipy.Spotify(auth=token)

In [121]:
# A function to extract track names and URIs from a playlist
def get_playlist_info(username, playlist_uri):
    # initialize vars
    offset = 0
    tracks, uris, names, artists = [], [], [], []

    # get playlist id and name from URI
    playlist_id = playlist_uri.split(':')[2]
    playlist_name = sp.user_playlist(username, playlist_id)['name']

    # get all tracks in given playlist (max limit is 100 at a time --> use offset)
    while True:
        results = sp.user_playlist_tracks(username, playlist_id, offset=offset)
        tracks += results['items']
        if results['next'] is not None:
            offset += 100
        else:
            break
        
    # get track metadata
    for track in tracks:
        names.append(track['track']['name'])
        artists.append(track['track']['artists'][0]['name'])
        uris.append(track['track']['uri'])
    
    return playlist_name, names, artists, uris

In [122]:
# Extract features from each track in a playlist
def get_features_for_playlist(df, username, uri):
  
    # get all track metadata from given playlist
    playlist_name, names, artists, uris = get_playlist_info(username, uri)
    
    # iterate through each track to get audio features and save data into dataframe
    for name, artist, track_uri in zip(names, artists, uris):
        # print(json.dumps(track_uri, indent=4))              
        # ^ DEBUG STATEMENT ^
        
        # access audio features for given track URI via spotipy 
        audio_features = sp.audio_features(track_uri)

        # get relevant audio features
        feature_subset = [audio_features[0][col] for col in df.columns if col not in ["name", "artist", "track_URI", "playlist"]]

        # compose a row of the dataframe by flattening the list of audio features
        row = [name, artist, track_uri, *feature_subset, playlist_name]
        df.loc[len(df.index)] = row
    return df

In [123]:
df = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])


In [124]:
df = get_features_for_playlist(df, user_config['username'], 'spotify:playlist:3VZCYCHOtahASqlkxE8vJx')
df


,name,artist,track_URI,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,playlist
0,Way With Words,Bahamas,spotify:track:6ioeN8jQ8Qc3tS8dhpoAVi,0.35900,0.785,0.530,0.001980,0.0774,-6.995,0.0634,146.380,0.7290,twenty
1,Caroline,Briston Maroney,spotify:track:3S5mohVxC0Xuj0tgZ7vU7g,0.72600,0.506,0.456,0.000037,0.1100,-10.635,0.0479,149.600,0.4060,twenty
2,Hear Me Calling,Juice WRLD,spotify:track:13ZyrkCDmRz5xY3seuAWYk,0.30800,0.699,0.687,0.000036,0.1210,-3.997,0.1060,88.932,0.4990,twenty
3,94 Bentley,SAINt JHN,spotify:track:0xxbDtdmc88FOV7YvpEtft,0.07920,0.696,0.558,0.000000,0.1620,-5.253,0.0512,113.000,0.2170,twenty
4,I Feel Like I'm Drowning,Two Feet,spotify:track:5h0ODSutX73Vt4vZsmmph0,0.70500,0.719,0.575,0.003340,0.0899,-5.436,0.0460,104.894,0.2350,twenty
...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,Lay Your Head On Me (feat. Marcus Mumford) - J...,Major Lazer,spotify:track:29v0kvBmnIurnEn34I2haX,0.00398,0.679,0.778,0.449000,0.2500,-6.063,0.0354,126.002,0.8760,twenty
445,Damn (feat. 6lack),Omah Lay,spotify:track:2SVObssPWzeJquNyR05NjL,0.32700,0.590,0.434,0.000000,0.1020,-5.975,0.2530,141.392,0.4930,twenty
446,Bubba,Jack Larsen,spotify:track:0qOe0oNsO7459dYDAlomiV,0.53300,0.285,0.539,0.003420,0.0995,-8.305,0.0273,170.007,0.0466,twenty
447,Perfect Blue,Elle Valenci,spotify:track:0u834tjHl0zB9xdPO5KrQX,0.89500,0.540,0.273,0.002230,0.1010,-12.123,0.0918,70.875,0.2860,twenty


In [18]:
sp.current_user_recently_played()

{'items': [{'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6UcJxoeHWWWyT5HZP064om'},
       'href': 'https://api.spotify.com/v1/artists/6UcJxoeHWWWyT5HZP064om',
       'id': '6UcJxoeHWWWyT5HZP064om',
       'name': 'Chet Faker',
       'type': 'artist',
       'uri': 'spotify:artist:6UcJxoeHWWWyT5HZP064om'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HR',
      'HU',
      'ID',
      'IE',
      'IL',
      'IS',
      'IT',
      'JO',
      'JP',
      'KW',
      'KZ',
      'LB',
      'LI',
      'LT',
   

In [19]:
sp.trace = True
user = sp.current_user()
pprint.pprint(user)

{'display_name': 'Sejal Dua',
 'external_urls': {'spotify': 'https://open.spotify.com/user/srrzaa61gi4hzx02avrosh1n9'},
 'followers': {'href': None, 'total': 24},
 'href': 'https://api.spotify.com/v1/users/srrzaa61gi4hzx02avrosh1n9',
 'id': 'srrzaa61gi4hzx02avrosh1n9',
 'images': [{'height': None,
             'url': 'https://i.scdn.co/image/ab6775700000ee85d34ef02977f41b325d7e54d3',
             'width': None}],
 'type': 'user',
 'uri': 'spotify:user:srrzaa61gi4hzx02avrosh1n9'}
